# Ejercicio Flask API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo. Ya tienes un modelo entrenado,desarrollarás una API que permita consumir dicho modelo desde cualquier otra tecnología.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web, comunicándose con una BBDD para ingestar o reentrenar el modelo. No vale base de datos en csv. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad. (/v2/predict)
2. Un endpoint para almacenar nuevos registros en la base de datos que deberás crear previamente.(/v2/ingest_data)
3. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan. (/v2/retrain)


**NOTAS**: 
1. Deberás desplegarlo desde un repositorio de github.
2. Ojo con la ruta para hacer el load de tu modelo y datos, comprueba cual es la ruta en la que está buscándolo.
3. El desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el desarrollo de una API con Flask.

**Entregable**: repositorio github.

In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import os
import pickle
from sklearn.linear_model import LinearRegression

app = Flask(__name__)
app.config['DEBUG'] = True
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///sales.db'
db = SQLAlchemy(app)

class Sales(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    tv = db.Column(db.Float, nullable=False)
    radio = db.Column(db.Float, nullable=False)
    newspaper = db.Column(db.Float, nullable=False)
    sales = db.Column(db.Float, nullable=False)

@app.route('/v2/predict', methods=['GET'])
def predict():
    model = pickle.load(open('advertising_model.pkl','rb'))

    tv = request.args.get('tv', None)
    radio = request.args.get('radio', None)
    newspaper = request.args.get('newspaper', None)

    if tv is None or radio is None or newspaper is None:
        return "Missing args, the input values are needed to predict"
    else:
        prediction = model.predict([[float(tv),float(radio),float(newspaper)]])
        return "The prediction of sales investing that amount of money in TV, radio and newspaper is: " + str(round(prediction[0],2)) + 'k €'

@app.route('/v2/ingest_data', methods=['POST'])
def ingest_data():
    data = request.get_json()
    new_sale = Sales(tv=data['tv'], radio=data['radio'], newspaper=data['newspaper'], sales=data['sales'])
    db.session.add(new_sale)
    db.session.commit()
    return jsonify({'message': 'New data ingested successfully'})

@app.route('/v2/retrain', methods=['GET'])
def retrain():
    data = Sales.query.all()
    X = [[d.tv, d.radio, d.newspaper] for d in data]
    y = [d.sales for d in data]

    model = LinearRegression()
    model.fit(X, y)

    pickle.dump(model, open('advertising_model.pkl', 'wb'))

    return jsonify({'message': 'Model retrained successfully'})

if __name__ == '__main__':
    app.run()